In [1]:
import torch
import numpy as np
import torchvision
from torchvision.models.detection import MaskRCNN

In [2]:
# Load the pre-trained Mask R-CNN model
model = torchvision.models.detection.maskrcnn_resnet50_fpn(pretrained=True)

/home2/rishabhk/apps/miniconda3/envs/NLP/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home2/rishabhk/apps/miniconda3/envs/NLP/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MaskRCNN_ResNet50_FPN_Weights.COCO_V1`. You can also use `weights=MaskRCNN_ResNet50_FPN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [3]:
# Load the pre-trained Mask R-CNN model
model = torchvision.models.detection.maskrcnn_resnet50_fpn(pretrained=True)

# Remove the mask prediction branch
del model.roi_heads.mask_head
del model.roi_heads.mask_predictor
del model.roi_heads.mask_roi_pool

# Replace the mask_roi_pool layer with roi_pool layer
model.roi_heads.mask_roi_pool = model.roi_heads.box_roi_pool

# Replace the output layer with a new layer for your specific task
num_classes = 40 # replace with the number of classes in your dataset
in_features = model.roi_heads.box_predictor.cls_score.in_features
model.roi_heads.box_predictor = torchvision.models.detection.faster_rcnn.FastRCNNPredictor(in_features, num_classes)

# Freeze the remaining layers
for param in model.parameters():
    param.requires_grad = False
model.roi_heads.box_predictor.cls_score.requires_grad = True
model.roi_heads.box_predictor.bbox_pred.requires_grad = True

In [ ]:
class ImagePixelDataset(Dataset):
    def __init__(self, root_dir, image_folder, label_folder, transform=None):
        self.root_dir = root_dir
        self.image_folder = image_folder
        self.label_folder = label_folder
        self.transform = transform
        self.image_list = os.listdir(os.path.join(root_dir, image_folder))
        self.label_list = os.listdir(os.path.join(root_dir, label_folder))

    def __len__(self):
        return len(self.image_list)

    def __getitem__(self, idx):
        image_path = os.path.join(self.root_dir, self.image_folder, self.image_list[idx])
        label_path = os.path.join(self.root_dir, self.label_folder, self.label_list[idx])
        image = Image.open(image_path)
        label = Image.open(label_path)

        if self.transform:
            image = self.transform(image)
            label = self.transform(label)

        return image, label

In [ ]:
root_dir = '../../cv_corpus/nyudv2/'
image_folder = 'hha'
label_folder = 'labels'

transform = ToTensor()

dataset = ImagePixelDataset(root_dir, image_folder, label_folder, transform=transform)

dataloader = DataLoader(dataset, batch_size=1, shuffle=False)

In [ ]:
# Train the model
for epoch in range(5):  # Number of epochs to train for
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # Get the inputs and labels
        inputs, labels = data

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)

        # Compute the loss
        loss = criterion(outputs['out'], labels)

        # Backward pass and optimize
        loss.backward()
        optimizer.step()

        # Print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:  # Print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' % (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished training')


In [ ]:
torch.save(model, "./finetuned_rcnn.pt")